## Filter
One of the most powerful parameters when defining a search is the ability to filter the results.  While a Query is more of an unstructured approach to search for text within documents, a Filter provides a well-defined structured approach when we know which properties we want to restrict our matching documents.

The following examples will be using some of the popular properties as outlined within the "Properties & Metadata" notebook.  In addition, the selection of these properties was determined based on the discovery techniques used such as metadata and debugging.

#### NuGet Packages

In [1]:
#r "nuget:Refinitiv.Data.Content, 1.0.0-beta4"
#r "nuget:Microsoft.Data.Analysis"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed Packages Microsoft.Data.Analysis, 0.20.1 Refinitiv.Data.Content, 1.0.0-beta4

Loading extensions from `C:\Users\U8007876\.nuget\packages\microsoft.data.analysis\0.20.1\interactive-extensions\dotnet\Microsoft.Data.Analysis.Interactive.dll`

In [2]:
using Newtonsoft.Json.Linq;
using Refinitiv.Data.Content.SearchService;
using Refinitiv.Data.Core;
using Microsoft.Data.Analysis;
using Microsoft.AspNetCore.Html;
using System.Data;
using System;
using Microsoft.DotNet.Interactive.Formatting;
using static Microsoft.DotNet.Interactive.Formatting.PocketViewTags;

#### Table Output
Helper routine to output data in a table format.

In [3]:
Formatter.Register<DataTable>((dt, writer) =>
{
    if (dt.Columns.Count > 0)
    {
        var rows = new List<ICollection<IHtmlContent>>();

        // Process Each data row
        foreach(var row in dt.Rows.Cast<DataRow>())
        {
            var cells = new List<IHtmlContent>();
            foreach (var cell in row.ItemArray)
            {
                switch (cell)
                {
                    case null:
                        break;
                    case JValue val:
                        cells.Add(td(val.ToString()));
                        break;
                    case IEnumerable<JToken> list:
                        cells.Add(td($"[{string.Join(", ", list.Select(node => $"{node}"))}]"));
                        break;
                    default:
                        cells.Add(td(cell));
                        break;
                }                
            }
            rows.Add(cells);
        }
        
        // Header
        var headers = new List<IHtmlContent>();
        foreach (DataColumn col in dt.Columns)
            headers.Add(th(col.ColumnName) as IHtmlContent);

        var t = table(thead(headers), tbody(rows.Select(r => tr(r))));
        writer.Write(t);
    }
}, "text/html");

In [4]:
// Create a session into the desktop
DesktopSession.Definition().GetSession().OnState((s, state, msg) => Console.WriteLine($"{DateTime.Now}:{msg}. (State: {state})"))
                                        .OnEvent((s, eventCode, msg) => Console.WriteLine($"{DateTime.Now}:{msg}. (Event: {eventCode})"))
                                        .Open();

2023-02-21 4:00:17 PM:DesktopSession. (State: DesktopSession is Pending)
2023-02-21 4:00:19 PM:DesktopSession. (Event: {
  "Contents": "Desktop Session Successfully Authenticated"
})
2023-02-21 4:00:19 PM:DesktopSession. (State: DesktopSession is Opened)


#### Example - Query vs Filter
As you become more familiar with Search, you will realize there are a number of different paths you can take to arrive at the same destination.

In [5]:
var response = Search.Definition(Search.View.Instruments).Query("US046353AQ14")
                                                         .Select("DocumentTitle, RIC, ISIN, CUSIP, SEDOL, PermID")
                                                         .GetData();
response.Data.Table

"AstraZeneca PLC, Plain Vanilla Fixed Coupon Bond, TRACE Eligible AZN 2.375 12-Jun-2022",US046353AQ14,046353AQ1,BF4SNM7,46645702869


In [6]:
// Retrieve the document(s) for the same ISIN using a Filter
response = Search.Definition(Search.View.Instruments).Filter("ISIN eq 'US046353AQ14'")
                                                     .Select("DocumentTitle, RIC, ISIN, CUSIP, SEDOL, PermID")
                                                     .GetData();
response.Data.Table

"AstraZeneca PLC, Plain Vanilla Fixed Coupon Bond, TRACE Eligible AZN 2.375 12-Jun-2022",US046353AQ14,046353AQ1,BF4SNM7,46645702869


In [7]:
// If you are provided a code but are not sure if the code is an ISIN, SEDOL, CUSIP etc, 
// try using the 'CodeResolutionBase' property.
// Note: I'm using the CUSIP for the same instrument within the Filter
response = Search.Definition(Search.View.Instruments).Filter("CodeResolutionBase eq '046353AQ1'")
                                                     .Select("DocumentTitle, RIC, ISIN, CUSIP, SEDOL, PermID")
                                                     .GetData();
response.Data.Table

"AstraZeneca PLC, Plain Vanilla Fixed Coupon Bond, TRACE Eligible AZN 2.375 12-Jun-2022",US046353AQ14,046353AQ1,BF4SNM7,46645702869


#### Example - Apple Equities

In [8]:
// Within the view (EquityQuotes), select all documents where the term 'AAPL' is within the ticker symbol.  For each, 
// ensure the type of asset is an 'equity' and that each asset is in an active state.  I chose to filter out the 
// AssetState of 'DC' (De-activated) as my criteria.
//
// Note: An alternative to "AssetType eq 'equity'" could be to use: "RCSAssetCategoryLeaf eq 'Ordinary Share'"
var filter = "AssetType eq 'equity' and AssetState ne 'DC' and TickerSymbol eq 'AAPL'";
var select = "RIC, CompositeDescriptiveName, ExchangeCode, ExchangeName, AssetType";

response = Search.Definition(Search.View.EquityQuotes).Filter(filter)
                                                      .Select(select)
                                                      .GetData();
response.Data.Table

AAPL.O,Apple Ord Shs APPLE ORD,NSQ,NASDAQ Global Select Consolidated,[EQUITY]
AAPL.OQ,Apple Ord Shs APPLE ORD,NSM,NASDAQ Stock Exchange Global Select Market,[EQUITY]
AAPL.DG,Apple Ord Shs APPLE ORD,GCD,Cboe EDGX Exchange - Nasdaq Global Select Market,[EQUITY]
AAPL.Z,Apple Ord Shs APPLE ORD APPLE INC,GCZ,Cboe BZX Exchange - Nasdaq Global Select Market,[EQUITY]
AAPL.B,Apple Ord Shs APPLE ORD,GSB,Boston SE when trading NASDAQ Global Select Market,[EQUITY]
AAPL.CCP,APPLE CAD HEDGED CDR Apple CAD Hedged CDR,CAQ,Canadian Composite Quote/Trade,[EQUITY]
AAPLE.MI,Apple Ord Shs APPLE ORD,MIL,Milan Stock Exchange,[EQUITY]
AAPL.DY,Apple Ord Shs APPLE ORD,GDA,Cboe EDGA Exchange - Nasdaq Global Select Market,[EQUITY]
AAPL.PH,Apple Ord Shs APPLE ORD APPLE INC,GSX,NASDAQ OMX PHLX Global Select Market,[EQUITY]
AAPL.ZY,Apple Ord Shs APPLE ORD APPLE INC,GCY,Cboe BYX Exchange - Nasdaq Global Select Market,[EQUITY]


#### Example - Vessels
List all Very Large Gas Containers (VLGC) that are in service within the last week.

In [9]:
// Build date range expression for the location of the vessel for the last 7 days
var date7DaysAgo = DateTime.Now.AddDays(-7);
var dateRange = @$"LocationTimestamp ge {date7DaysAgo:yyyy-MM-dd} and LocationTimestamp lt {DateTime.Now:yyyy-MM-dd}";
dateRange

LocationTimestamp ge 2023-02-14 and LocationTimestamp lt 2023-02-21

In [10]:
// Show the location of each vessel and some general details such as location, destination and coordinates.
filter = $"RCSAssetTypeLeaf eq 'VLGC' and VesselStatus eq 'IN SERVICE' and {dateRange}";
select = "RIC, SynonymSearchAll, DTSource, LocationTimestamp, IMO,Latitude, Longitude, LocationDestination";

response = Search.Definition(Search.View.VesselPhysicalAssets).Filter(filter)
                                                              .Select(select)
                                                              .Top(100)
                                                              .GetData();
response.Data.Table

C}KY7309484995,Indian India Liquefied Petroleum Gas Very Large Gas Carrier,Arabia Sea,2023-02-19 12:29:16 PM,9264908,18.850416183472,64.966377258301,RUWAIS [Jebel Dhanna/Ruwais]
C}KY7309693658,Japanese Liquefied Petroleum Gas Very Large Gas Carrier,Pacific Ocean (Central America)|Hawaii Tanker Zone,2023-02-19 1:45:59 AM,9723679,20.646778106689,-170.770477294922,PA BLB [Balboa]
C}KY7309843282,Chinese Liquefied Petroleum Gas Very Large Gas Carrier,South China Sea|Canton Tanker Zone,2023-02-20 10:58:28 PM,9800506,21.52293586731,118.206092834473,CNQZH [Qinzhou]
C}KY7309432434,Chinese Vietnamese Liquefied Petroleum Gas Very Large Gas Carrier,South China Sea|Canton Tanker Zone,2023-02-20 10:52:54 PM,7420089,20.458866119385,106.725936889648,HAI HA BUOY


#### Example - Floating Rate Notes
Retrieve all floating rate notes (floating coupon) in the healthcare sector which are issued in Euros.  For each, ensure the coupon rate is great than 2 and our Moody's rating is greater than 'Baa1'.

To determine the rating, we apply the following logic utilizing the RatingsScope property - which is a nested property providing additional details:

- RatingsScope(RatingType eq 'MIS' and CurrentRatingRank lt 10)

The RatingType of 'MIS' refers to the "Moody's Long-term Issuer Rating".
The CurrentRatingRank < 10 instructs search to look for ratings less than 10 ('Baa1').  **Note**: The lower the rank, the better the rating.

In [11]:
filter = @"CouponRate gt 2 and
           RatingsScope(RatingType eq 'MIS' and CurrentRatingRank lt 10) and
           RCSCouponTypeName eq 'Floating Coupon' and
           IsActive eq true and
           RCSTRBC2012Name eq 'Healthcare' and
           RCSCurrencyLeaf xeq 'Euro'";
select = "CouponCurrency, IssuerName, IssueDate, RIC, CouponRate, ISIN, Ticker, RCSTRBC2012Leaf, MaturityDate, AssetState";

response = Search.Definition(Search.View.GovCorpInstruments).Filter(filter)
                                                            .Select(select)
                                                            .GetData();
response.Data.Table

EUR,Sanofi FR,1983-07-01 12:00:00 AM,FR0000140022=,15.5,FR0000140022,SASY,Pharmaceuticals (NEC)
EUR,Sanofi FR,1987-10-01 12:00:00 AM,FR0000047656=MFID,15.5,FR0000047656,SASY,Pharmaceuticals (NEC)


#### Example - Outstanding Green Bonds
List the outstanding Green Bonds for Telefonica.

In [12]:
select = @"ssuerName, DocumentTitle, RIC, CouponRate, ISIN, IssuerTicker, DebtTypeDescription, 
           AmountOutstanding, FaceOutstanding";

response = Search.Definition(Search.View.GovCorpInstruments).Filter("IssuerName eq 'Telefonica' and IsGreenBond eq true")
                                                            .Select(select)
                                                            .GetData();
response.Data.Table

"Telefonica Europe BV, Bond, TEF 2.502",ES210981985=,2.502,XS2109819859,TEFBV,Junior Subordinated Note,500000000
"Telefonica Europe BV, Bond, TEFBV 7.125",ES246260567=,7.125,XS2462605671,TEFBV,Junior Subordinated Note,750000000
"Telefonica Europe BV, Bond, TEFBV 6.135",ES258238915=,6.135,XS2582389156,TEFBV,Capital Security,1000000000
"Telefonica Emisiones SAU, Plain Vanilla Fixed Coupon Bond, TEF 1.069 05-Feb-2024",ES194600445=,1.069,XS1946004451,TEFEMI,Senior Note,1000000000


#### Example - Equity Index listing in the U.S.A

In [13]:
// Based on the Asset Category (Equity Index), list the top 10 indexes within the USA.
filter = "RCSIndexCountryGroupLeaf xeq 'United States' and RCSAssetCategoryLeaf xeq 'Equity Index'";

response = Search.Definition(Search.View.IndexInstruments).Filter(filter)
                                                          .Select("DocumentTitle, RIC, RCSTRBC2012Leaf")
                                                          .GetData();
response.Data.Table

"NASDAQ 100 Index, Equity Index",.NDX,UK Investment Trusts
"NASDAQ Composite Index, Equity Index",.IXIC,Financial & Commodity Market Operators & Service Providers (NEC)
"Dow Jones Industrial Average Index, Equity Index",.DJI,Consumer Publishing (NEC)
"Refinitiv United States Price Return Index, Equity Index",.TRXFLDUSP,Professional Information Services (NEC)
"S&P 500 Index - CBOE, Equity Index",.SPX,Magazine Publishing
"NYSE Composite Index, Equity Index",.NYA,Financial & Commodity Market Operators & Service Providers (NEC)
"Dow Jones Transport Average Index, Equity Index",.DJT,Consumer Publishing (NEC)
"Dow Jones Composite Index, Equity Index",.DJA,Consumer Publishing (NEC)
"Dow Jones Utility Average Index, Equity Index",.DJU,Consumer Publishing (NEC)
"Philadelphia SE Semiconductor Index, Equity Index",.SOX,Securities & Commodity Exchanges


#### Example - Search for Brazil global sovereign bonds in USD with a duration greater than 1

In [14]:
filter = @"RCSIssuerCountryLeaf eq 'Brazil' and
           RCSCurrencyLeaf eq 'US Dollar' and
           RCSTRBC2012Leaf eq 'Government & Government Finance' and
           OptionAdjustedDuration gt 1";
select = "RIC, ISIN, MaturityDate, CouponRate, IndustrySector, FaceOutstandingUSD, OptionAdjustedDuration";

response = Search.Definition(Search.View.FixedIncomeInstruments).Filter(filter)
                                                                .Select(select)
                                                                .GetData();
response.Data.Table

BR012631375=,US105756AR10,2024-04-15 12:00:00 AM,8.875,SOVERGRN,1029030000,1.055949
BR017244388=,US105756AZ36,2024-04-15 12:00:00 AM,8.875,SOVERGRN,85557000,1.055949
105756BV1=,US105756BV13,2025-01-07 12:00:00 AM,4.25,SOVERGRN,4300000000,1.765833
BR021210641=,US105756BF62,2025-02-04 12:00:00 AM,8.75,SOVERGRN,688201000,1.785662
105756CD0=,US105756CD06,2025-06-06 12:00:00 AM,2.875,SOVERGRN,1750000000,2.156112
105756BX7=,US105756BX78,2026-04-07 12:00:00 AM,6,SOVERGRN,2176245000,2.76905
105756AE0=,US105756AE07,2027-05-15 12:00:00 AM,10.125,SOVERGRN,765023000,3.426415
105756BZ2=,US105756BZ27,2028-01-13 12:00:00 AM,4.625,SOVERGRN,3000000000,4.216577
105756CA6=,US105756CA66,2029-05-30 12:00:00 AM,4.5,SOVERGRN,2000000000,5.250205
BR010862957=,US105756AL40,2030-03-06 12:00:00 AM,12.25,SOVERGRN,237592000,4.894292


In [15]:
// Alternatively, you have the option of trying it this way which will render the same results
filter = @"IssuerCountryName eq 'Brazil' and
           OptionAdjustedDuration gt 1 and
           Currency eq 'USD' and IndustrySectorDescription eq 'Sovereign'";
select = "RIC, ISIN, MaturityDate, CouponRate, IndustrySector, FaceOutstandingUSD, OptionAdjustedDuration";

response = Search.Definition(Search.View.FixedIncomeInstruments).Filter(filter)
                                                                .Select(select)
                                                                .GetData();
response.Data.Table

BR012631375=,US105756AR10,2024-04-15 12:00:00 AM,8.875,SOVERGRN,1029030000,1.055949
BR017244388=,US105756AZ36,2024-04-15 12:00:00 AM,8.875,SOVERGRN,85557000,1.055949
105756BV1=,US105756BV13,2025-01-07 12:00:00 AM,4.25,SOVERGRN,4300000000,1.765833
BR021210641=,US105756BF62,2025-02-04 12:00:00 AM,8.75,SOVERGRN,688201000,1.785662
105756CD0=,US105756CD06,2025-06-06 12:00:00 AM,2.875,SOVERGRN,1750000000,2.156112
105756BX7=,US105756BX78,2026-04-07 12:00:00 AM,6,SOVERGRN,2176245000,2.76905
105756AE0=,US105756AE07,2027-05-15 12:00:00 AM,10.125,SOVERGRN,765023000,3.426415
105756BZ2=,US105756BZ27,2028-01-13 12:00:00 AM,4.625,SOVERGRN,3000000000,4.216577
105756CA6=,US105756CA66,2029-05-30 12:00:00 AM,4.5,SOVERGRN,2000000000,5.250205
BR010862957=,US105756AL40,2030-03-06 12:00:00 AM,12.25,SOVERGRN,237592000,4.894292


#### Example - Top executives

In [16]:
// Using the People View, we can utilize the OfficerDirector property, which contains a number of nested or sub-properties 
// that allow me to perform a more fine-grained search against active people that have been assigned a title code of 
// CEO (Chief Executive Officer). The results are listed based on a default rank defined by the search service.
filter = "OfficerDirector(RoleTitleCode eq 'CEO' and RoleStatus eq 'Active')";
select = @"FullName, PrimaryTitleAsReported, PrimaryOfficerDirector, RIC, 
           OfficerDirectorActiveCompanies, PrimaryOfficerDirectorRIC";
           
response = Search.Definition(Search.View.People).Filter(filter)
                                                .Select(select)
                                                .Top(20)
                                                .GetData();
response.Data.Table

Satya Nadella,"Chairman of the Board, Chief Executive Officer",Microsoft Corp,"[Starbucks Corp, Chandler Insurance Company Ltd, Microsoft Corp]",MSFT.O
Tim D. Cook,"Chief Executive Officer, Director",Apple Inc,"[Apple Inc, Nike Inc]",AAPL.O
Andrew R. Jassy,"President, Chief Executive Officer, Director",Amazon.com Inc,"[Coupa Software Inc, Amazon.com Inc]",AMZN.O
Sundar Pichai,"Chief Executive Officer, Director",Alphabet Inc,"[Alphabet Inc, Ruba Inc (Fka Kudo Inc)]",GOOGL.O
R. Martin Chavez,Director,Alphabet Inc,"[Alphabet Inc, Quorum Software Systems Inc, Kiodex Inc]",GOOGL.O
Mark Zuckerberg,"Chairman of the Board, Chief Executive Officer, Founder",Meta Platforms Inc,"[Chan Zuckerberg Initiative LLC, Meta Platforms Inc]",META.O
Warren E. Buffett,"Chairman of the Board, Chief Executive Officer",Berkshire Hathaway Inc,"[Grinnell College, Bill & Melinda Gates Foundation Trust, Berkshire Hathaway Inc, Kraft Heinz Intermediate Corporation II, Kraft Heinz Foods Co]",BRKa
Daniel Zhang,"Chairman of the Board, Chief Executive Officer",Alibaba Group Holding Ltd,[Alibaba Group Holding Ltd],BABA.K
Huateng Ma,"Executive Chairman of the Board, Chief Executive Officer",Tencent Holdings Ltd,"[Huarong Tianze Investment Co Ltd, Tencent Holdings Ltd, Tencent Technology Beijing Co Ltd]",0700.HK
Ryan McInerney,"Chief Executive Officer, Director",Visa Inc,[Visa Inc],V
Joaquin Boix Duato,"Chairman of the Board, Chief Executive Officer",Johnson & Johnson,[Johnson & Johnson],JNJ


#### Example - Live Options
The following example will specify an underlying instrument (RIC) and pull the list of options that are active, i.e. not expired.

In [17]:
// Define our underlying
var underlying = "AAPL.O";

// To search for our instrument, I'm utilizing the property: 'UnderlyingQuoteRIC'.  The property does not support an 
// exact match 'xeq', so we must use the partial match 'eq' to find all hits. Depending on the instrument you specify, 
// this may introduce additional hits.  For example, if you are using the underlying 'AAPL.O', search will also find a 
// hit on 'AAPL.OQ', for example. To protect against this, we can utilize a combination of the built-in functions, 
// 'startswith' and 'endwith' to perform our match.
filter = @$"AssetState eq 'AC' and
            startswith(UnderlyingQuoteRIC,'{underlying}') and
            endswith(UnderlyingQuoteRIC,'{underlying}') and
            RCSAssetClass eq 'OPT' and IsChain eq false and
            ExpiryDate ge {DateTime.Now:yyyy-MM-dd}";
            
select = "DocumentTitle, RIC, ExpiryDate, CallPutOption, StrikePrice";
           
response = Search.Definition(Search.View.EquityDerivativeQuotes).Filter(filter)
                                                                .Select(select)
                                                                .Top(10000)
                                                                .GetData();
response.Data.Table

"OPRA Apple Equity Option 150 Call Mar 2023 , Equity Cash Option, Call 150 USD 17-Mar-2023, OPRA",AAPLC172315000.U,2023-03-17 12:00:00 AM,Call,150
"OPRA Apple Equity Option 155 Call Mar 2023 , Equity Cash Option, Call 155 USD 17-Mar-2023, OPRA",AAPLC172315500.U,2023-03-17 12:00:00 AM,Call,155
"OPRA Apple Equity Option 160 Call Mar 2023 , Equity Cash Option, Call 160 USD 17-Mar-2023, OPRA",AAPLC172316000.U,2023-03-17 12:00:00 AM,Call,160
"OPRA Apple Equity Option 145 Put Mar 2023 , Equity Cash Option, Put 145 USD 17-Mar-2023, OPRA",AAPLO172314500.U,2023-03-17 12:00:00 AM,Put,145
"OPRA Apple Equity Option 165 Call Mar 2023 , Equity Cash Option, Call 165 USD 17-Mar-2023, OPRA",AAPLC172316500.U,2023-03-17 12:00:00 AM,Call,165
"OPRA Apple Equity Option 150 Put Mar 2023 , Equity Cash Option, Put 150 USD 17-Mar-2023, OPRA",AAPLO172315000.U,2023-03-17 12:00:00 AM,Put,150
"OPRA Apple Equity Option 170 Call Mar 2023 , Equity Cash Option, Call 170 USD 17-Mar-2023, OPRA",AAPLC172317000.U,2023-03-17 12:00:00 AM,Call,170
"OPRA Apple Equity Option 130 Put Mar 2023 , Equity Cash Option, Put 130 USD 17-Mar-2023, OPRA",AAPLO172313000.U,2023-03-17 12:00:00 AM,Put,130
"OPRA Apple Equity Option 140 Put Mar 2023 , Equity Cash Option, Put 140 USD 17-Mar-2023, OPRA",AAPLO172314000.U,2023-03-17 12:00:00 AM,Put,140
"OPRA Apple Equity Option 145 Call Mar 2023 , Equity Cash Option, Call 145 USD 17-Mar-2023, OPRA",AAPLC172314500.U,2023-03-17 12:00:00 AM,Call,145
"OPRA Apple Equity Option 135 Put Mar 2023 , Equity Cash Option, Put 135 USD 17-Mar-2023, OPRA",AAPLO172313500.U,2023-03-17 12:00:00 AM,Put,135
